# 算法模型

In [1]:
import argparse
import datetime
import os

import tensorflow as tf
from absl import logging

import trainer.constants as cst
import trainer.data_pipeline as dp
import trainer.split_model as split_model
import trainer.full_cnn_model as full_cnn_model
from trainer.callbacks import CustomCheckpoints


## 导入模型函数

In [2]:
def get_args():
    """Argument parser.

    Returns:
        Dictionary of arguments.
    """
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--job-dir',
        type=str,
        default=cst.BASE_DIR,
        help='local or GCS location for writing checkpoints and exporting models')
    parser.add_argument(
        '--data-dir-train',
        type=str,
        default=cst.TRAIN_SET,
        help='local or GCS location for reading TFRecord files for the training set')
    parser.add_argument(
        '--data-dir-validate',
        type=str,
        default=cst.TEST_SET,
        help='local or GCS location for reading TFRecord files for the validation set')
    parser.add_argument(
        '--tboard-dir',         # no default so we can construct dynamically with timestamp
        type=str,
        help='local or GCS location for reading TensorBoard files')
    parser.add_argument(
        '--saved-model-dir',    # no default so we can construct dynamically with timestamp
        type=str,
        help='local or GCS location for saving trained Keras models')
    parser.add_argument(
        '--num-epochs',
        type=int,
        default=5,
        help='number of times to go through the data, default=3')
    parser.add_argument(
        '--batch-size',
        default=32,
        type=int,
        help='number of records to read during each training step, default=16')
    parser.add_argument(
        '--window-size',
        default=20,
        type=int,
        help='window size for sliding window in training sample generation, default=100')
    parser.add_argument(
        '--shift',
        default=5,
        type=int,
        help='shift for sliding window in training sample generation, default=20')
    parser.add_argument(
        '--stride',
        default=1,
        type=int,
        help='stride inside sliding window in training sample generation, default=1')
    parser.add_argument(
        '--verbosity',
        choices=['DEBUG', 'ERROR', 'FATAL', 'INFO', 'WARN'],
        default='DEBUG')
    parser.add_argument(
        '--loss',
        default='mean_squared_error',
        type=str,
        help='loss function used by the model, default=mean_squared_error')
    parser.add_argument(
        '--shuffle',
        default=True,
        type=bool,
        help='shuffle the batched dataset, default=True'
    )
    parser.add_argument(
        '--shuffle-buffer',
        default=5,
        type=int,
        help='Bigger buffer size means better shuffling but longer setup time. Default=500'
    )
    parser.add_argument(
        '--save-from',
        default=80,
        type=int,
        help='epoch after which model checkpoints are saved, default=80'
    )
    parser.add_argument(
        '--model',
        default='split_model',
        type=str,
        help='The type of model to use, default="split_model", options="split_model", "full_cnn_model"'
    )
    args, _ = parser.parse_known_args()
    return args


def train_and_evaluate(args, tboard_dir, hparams=None):
    """Trains and evaluates the Keras model.

    Uses the Keras model defined in model.py and trains on data loaded and
    preprocessed in data_pipeline.py. Saves the trained model in TensorFlow SavedModel
    format to the path defined in part by the --job-dir argument.

    Args:
    args: dictionary of arguments - see get_args() for details
    """
    # Config datasets for consistent usage
    ds_config = dict(window_size=args.window_size,
                     shift=args.shift,
                     stride=args.stride,
                     batch_size=args.batch_size)
    ds_train_path = args.data_dir_train
    ds_val_path = args.data_dir_validate

    # create model
    if args.model == 'split_model':
        print("Using split model!")
        model = split_model.create_keras_model(window_size=ds_config["window_size"],
                                               loss=args.loss,
                                               hparams_config=hparams)
    if args.model == 'full_cnn_model':
        print("Using full cnn model!")
        model = full_cnn_model.create_keras_model(window_size=ds_config["window_size"],
                                                  loss=args.loss,
                                                  hparams_config=hparams)

    # Calculate steps_per_epoch_train, steps_per_epoch_test
    # This is needed, since for counting repeat has to be false
    steps_per_epoch_train = calculate_steps_per_epoch(data_dir=ds_train_path, dataset_config=ds_config)

    steps_per_epoch_validate = calculate_steps_per_epoch(data_dir=ds_val_path, dataset_config=ds_config)

    # load datasets
    dataset_train = dp.create_dataset(data_dir=ds_train_path,
                                      window_size=ds_config["window_size"],
                                      shift=ds_config["shift"],
                                      stride=ds_config["stride"],
                                      batch_size=ds_config["batch_size"])

    dataset_validate = dp.create_dataset(data_dir=ds_val_path,
                                         window_size=ds_config["window_size"],
                                         shift=ds_config["shift"],
                                         stride=ds_config["stride"],
                                         batch_size=ds_config["batch_size"])

    # if hparams is passed, we're running a HPO-job
    if hparams:
        checkpoint_callback = CustomCheckpoints(save_last_only=True,
                                                log_dir=tboard_dir,
                                                dataset_path=ds_val_path,
                                                dataset_config=ds_config,
                                                save_eval_plot=False)
    else:
        checkpoint_callback = CustomCheckpoints(save_best_only=True,
                                                start_epoch=args.save_from,
                                                log_dir=tboard_dir,
                                                dataset_path=ds_val_path,
                                                dataset_config=ds_config,
                                                save_eval_plot=False)
    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=tboard_dir,
                                       histogram_freq=0,
                                       write_graph=False,
                                       ),
        checkpoint_callback,
    ]

    model.summary()

    # train model
    history = model.fit(
        dataset_train,
        epochs=args.num_epochs,
        steps_per_epoch=steps_per_epoch_train,
        validation_data=dataset_validate,
        validation_steps=steps_per_epoch_validate,
        verbose=2,
        callbacks=callbacks)

    mae_current = min(history.history["val_mae_current_cycle"])
    mae_remaining = min(history.history["val_mae_remaining_cycles"])
    print('\nhistory dict:', history.history)
    return mae_current, mae_remaining


def calculate_steps_per_epoch(data_dir, dataset_config):
    temp_dataset = dp.create_dataset(data_dir=data_dir,
                                     window_size=dataset_config["window_size"],
                                     shift=dataset_config["shift"],
                                     stride=dataset_config["stride"],
                                     batch_size=dataset_config["batch_size"],
                                     repeat=False)
    steps_per_epoch = 0
    for batch in temp_dataset:
        steps_per_epoch += 1
    return steps_per_epoch


def get_tboard_dir():
    run_timestr = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    if args.tboard_dir is None:
        tboard_dir = os.path.join(cst.TENSORBOARD_DIR, "jobs", run_timestr)
    else:
        tboard_dir = args.tboard_dir
    return tboard_dir

## 启动训练脚本

In [3]:
if __name__ == '__main__':
    args = get_args()
    logging.set_verbosity(args.verbosity)
    train_and_evaluate(args, get_tboard_dir())



Using split model!
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Qdlin (InputLayer)              [(None, 20, 1000, 1) 0                                            
__________________________________________________________________________________________________
Tdlin (InputLayer)              [(None, 20, 1000, 1) 0                                            
__________________________________________________________________________________________________
detail_concat (Concatenate)     (None, 20, 1000, 2)  0           Qdlin[0][0]                      
                                                                 Tdlin[0][0]                      
__________________________________________________________________________________________________
convolution (TimeDistributed)   (None, 20, 334, 32)  608         detail_con

## 模型预测
### 加载预测及可视化函数

In [7]:
import numpy as np
import json
import tensorflow as tf
from trainer.plot import plot_single_prediction
from trainer.custom_metrics_losses import mae_current_cycle, mae_remaining_cycles
from trainer.clippy import Clippy, clipped_relu

def make_plot(model, predictions):
    predictions = np.array(predictions)
    first_pred = predictions[0]
    window_size = model.input_shape[0][1]
    # this number comes from training dataset preprocessing
    scaling_factors_dict = {"Remaining_cycles": 2159.0}
    mean_cycle_life = 674  # calculated from training set
    figure = plot_single_prediction(first_pred,
                                  window_size,
                                  scaling_factors_dict,
                                  mean_cycle_life)
    return figure

def make_prediction(model, cycle_data):
    cycles = { 'Qdlin': np.array(json.loads(cycle_data['Qdlin'])),
                'Tdlin': np.array(json.loads(cycle_data['Tdlin'])),
                'IR': np.array(json.loads(cycle_data['IR'])),
                'Discharge_time': np.array(json.loads(cycle_data['Discharge_time'])),
                'QD': np.array(json.loads(cycle_data['QD']))
            }

    predictions = model.predict(cycles)
    print_predictions = model.predict(cycles)
    print_predictions[0] = print_predictions[0] * 2159
    print("Returning predictions:")
    print(print_predictions)
    return predictions

### 导入模型

In [20]:
MODEL_DIR = "saved_model/"
# model = tf.keras.models.load_model(MODEL_DIR, custom_objects={'clippy': Clippy(clipped_relu),
#                                                               'mae_current_cycle': mae_current_cycle,
#                                                               'mae_remaining_cycles': mae_remaining_cycles})
#
model = tf.keras.experimental.load_from_saved_model(MODEL_DIR,
                                       custom_objects={'clippy': Clippy(clipped_relu),
                                                       'mae_current_cycle': mae_current_cycle,
                                                       'mae_remaining_cycles': mae_remaining_cycles})

### 运用模型预测

In [23]:
json_file_sample = './sample_input_4.json'

with open(json_file_sample, 'r') as infd:
  json_data = json.load(infd)
#print(json_data)
predictions_soh_rul= make_prediction(model, json_data)

Returning predictions:
[[257.15192 605.04913]]


In [24]:
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
figure = make_plot(model, predictions_soh_rul)
pyo.plot(figure,filename='Prediction.html')
print('You can click the Prediction.html in the left sidebar to see the prediction')

You can click the Prediction.html to see the prediction
